In [1]:
import os
import shutil
from pyspark.sql import SparkSession

In [5]:
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.getOrCreate()

23/11/05 16:38:02 WARN Utils: Your hostname, frank-xps resolves to a loopback address: 127.0.1.1; using 192.168.1.63 instead (on interface wlp0s20f3)
23/11/05 16:38:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 16:38:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/05 16:38:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path_dataset = '/media/frank/My Passport/datalab/datasets/big_dataset'
products_table = spark.read.parquet(path_dataset + "/products_parquet")
sales_table = spark.read.parquet(path_dataset + "/sales_parquet")
sellers_table = spark.read.parquet(path_dataset + "/sellers_parquet")


In [8]:
#   Print the number of orders
print("Number of Orders: {}".format(sales_table.count()))

#   Print the number of sellers
print("Number of sellers: {}".format(sellers_table.count()))

#   Print the number of products
print("Number of products: {}".format(products_table.count()))

Number of Orders: 20000040
Number of sellers: 10
Number of products: 75000000


In [11]:
sales_table.show()

+--------+----------+---------+----------+---------------+--------------------+
|order_id|product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|
+--------+----------+---------+----------+---------------+--------------------+
|       1|         0|        0|2020-07-10|             26|kyeibuumwlyhuwksx...|
|       2|         0|        0|2020-07-08|             13|jfyuoyfkeyqkckwbu...|
|       3|         0|        0|2020-07-05|             38|uyjihlzhzcswxcccx...|
|       4|         0|        0|2020-07-05|             56|umnxvoqbdzpbwjqmz...|
|       5|         0|        0|2020-07-05|             11|zmqexmaawmvdpqhih...|
|       6|         0|        0|2020-07-01|             82|lmuhhkpyuoyslwmvX...|
|       7|         0|        0|2020-07-04|             15|zoqweontumefxbgvu...|
|       8|         0|        0|2020-07-08|             79|sgldfgtcxufasnvsc...|
|       9|         0|        0|2020-07-10|             25|jnykelwjjebgkwgmu...|
|      10|         0|        0|2020-07-0

In [12]:
sales_table.groupBy(col("product_id")).agg(
    count("*").alias("cnt")).orderBy(col("cnt").desc()).show()

+----------+--------+
|product_id|     cnt|
+----------+--------+
|         0|19000000|
|  32602520|       3|
|  69790381|       3|
|  56011040|       3|
|  31136332|       3|
|  61540351|       3|
|  28592106|       3|
|  17944574|       3|
|  36269838|       3|
|  14542470|       3|
|  35669461|       3|
|   2839667|       3|
|  40496308|       3|
|  19978383|       3|
|   2316238|       3|
|  18182299|       3|
|  28183035|       3|
|  52606213|       3|
|  40579633|       3|
|  34681047|       3|
+----------+--------+
only showing top 20 rows



In [19]:
product_seller_count = sales_table \
    .groupBy(sales_table.product_id, sales_table.seller_id) \
    .agg(count(sales_table.order_id).alias('times_sold')) \
    .orderBy(col("times_sold").desc())

In [21]:
product_seller_count.count()

999276

In [20]:
product_seller_count.show()

+----------+---------+----------+
|product_id|seller_id|times_sold|
+----------+---------+----------+
|         0|        0|  19000000|
|  31235092|        5|         2|
|  61570737|        4|         2|
|  36718513|        5|         2|
|  15149556|        1|         2|
|  33166517|        6|         2|
|  31902552|        7|         2|
|   4863744|        1|         2|
|  35069646|        8|         2|
|  10432387|        9|         2|
|  20762138|        9|         2|
|   4819033|        3|         2|
|  69979310|        3|         2|
|  46112802|        7|         2|
|  62508894|        7|         2|
|  12181569|        3|         2|
|  61787941|        4|         2|
|  73195190|        8|         2|
|    933496|        7|         2|
|  21148738|        6|         2|
+----------+---------+----------+
only showing top 20 rows



In [22]:
popular_products = product_seller_count.filter(col('times_sold') >= 2)

In [23]:
popular_products.count()

766

In [24]:
popular_products.show()

+----------+---------+----------+
|product_id|seller_id|times_sold|
+----------+---------+----------+
|         0|        0|  19000000|
|  31235092|        5|         2|
|  35069646|        8|         2|
|  33166517|        6|         2|
|  61787941|        4|         2|
|   4863744|        1|         2|
|  70956858|        4|         2|
|  10432387|        9|         2|
|  63082689|        1|         2|
|   4819033|        3|         2|
|  71807028|        6|         2|
|  46112802|        7|         2|
|  58080735|        9|         2|
|  12181569|        3|         2|
|   4878806|        8|         2|
|  73195190|        8|         2|
|  27580157|        1|         2|
|  21148738|        6|         2|
|  73329635|        9|         2|
|  20400034|        3|         2|
+----------+---------+----------+
only showing top 20 rows

